1. Mount your Google Drive.

In [ ]:
# Mount Google Drive

from google.colab import drive

drive.mount('/content/gdrive')

2. (Optional) Check GPU

In [ ]:
# Check GPU (make sure runtimetype GPU (T4 or A100) is selected)
# If 'command not found' then change the runtimetype (click 'Runtime' and select change runtimetype in the drop down menu)

!nvidia-smi

3. Copy data.zip to Colab file system and unzip.

In [ ]:
# Prepare data

dataset = '/content/gdrive/MyDrive/hailo/vespA17000.zip' # @param {type:'string'}

!scp {dataset} '/content/dataset.zip'

!unzip '/content/dataset.zip'
!rm '/content/dataset.zip'

4. If the name of the folder with the data is not `dataset` then rename it.

In [ ]:
import os

# Rename the extracted folder to dataset
foldername = "/content/vespA_combined2024" # @param {type:'string'}
new_path = '/content/dataset'
os.rename(foldername, new_path)

5. Check if folderstructure is correct. The output should be:
```
data.yaml  test  train	valid
images	labels
images	labels
```

In [ ]:
# Check is path is correct

#  output should be
#     train val
#     images labels
#     dataset.yaml data.zip

# If not, change your datastructure as described under 1.

!ls '/content/dataset/'

!ls '/content/dataset/train/'

!ls '/content/dataset/valid/'



6. Install libraries and train the model

In [ ]:
#Installing ultralytics and wandb
!pip install -U ultralytics wandb

7. (Optional) Track with Weights and Biases.

  Ultralytics YOLO11 integration with Weights & Biases for enhanced experiment tracking, model-checkpointing, and visualization of model performance. For more information click Weights and Biasis

  Make account and a project in W&B.
  Store your API key in the Secrets in Google Colab In your Colab notebook, click the key icon in the left sidebar (this is the secrets manager).
  Add a secret with the name WANDB_API_KEY and paste your WandB API key as its value. You can find your api key here.

In [ ]:
import wandb
from ultralytics import YOLO
from google.colab import userdata

# Retrieve the API key from Colab secrets
wandb_api_key = userdata.get('WANDB_API_KEY')

# Initialize your Weights & Biases environment
wandb.login(key=wandb_api_key)

8. Train the model.


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8s.pt")  # load a pretained model (choose yolov8n for the smallest model)

# Train and Fine-Tune the Model
model.train(data="/content/dataset/data.yaml",
            epochs=200,
            batch=-1,
            patience=20,
            project="ultralytics",
            name="yolov8s"
            )

9. Export the model in ONNX format.

In [ ]:
from ultralytics import YOLO


# Export the model
model.export(format="onnx")

In [ ]:
import onnx

weights_path = "/content/ultralytics/yolov8s/weights/"  # @param {type:"string"}

# Load the ONNX model
model = onnx.load(weights_path + "best.onnx")

# Save with default opset (no explicit opset setting)
onnx.save(model, weights_path + "best_default_opset.onnx")

# Save with opset 11
model.opset_import[0].version = 11
onnx.save(model, weights_path + "best_opset11.onnx")

# Save with opset 9
model.opset_import[0].version = 9
onnx.save(model, weights_path + "best_opset9.onnx")

# Save with opset 8
model.opset_import[0].version = 8
onnx.save(model, weights_path + "best_opset8.onnx")


10. Zip and Download the results to your local computer

  ---=== Data is lost when closing the page!!! ===---

## If an error occurs:
1. Click `Runtime` in the toolbar on the top
2. Select `stop session` / `uitvoering onderbreken` from the drop down menu
3. Run this codeblock again with `Restart session` / `Sessie opnieuw starten`

In [ ]:
# DON'T FORGET TO DOWNLOAD THE RESULTS!!!

# Data is lost when closing the page!!!

# IN MOST CASES AN ERROR OCCURS:
# Click `Runtime`, select `Restart session` / `Sessie opnieuw opstarten`
# then run this codeblock again

from google.colab import files

try:
  !zip -r /content/runs.zip /content/ultralytics/
  files.download('/content/yolov8sModel.zip')
except Exception as e:
  print(f"An error occurred: {e}")
  print("Click 'Runtime' -> 'Restart session' and try running the code again.")